In [3]:
!!git clone https://github.com/ggerganov/llama.cpp

["Cloning into 'llama.cpp'...",
 'remote: Enumerating objects: 19505, done.\x1b[K',
 'remote: Counting objects:   0% (1/6819)\x1b[K',
 'remote: Counting objects:   1% (69/6819)\x1b[K',
 'remote: Counting objects:   2% (137/6819)\x1b[K',
 'remote: Counting objects:   3% (205/6819)\x1b[K',
 'remote: Counting objects:   4% (273/6819)\x1b[K',
 'remote: Counting objects:   5% (341/6819)\x1b[K',
 'remote: Counting objects:   6% (410/6819)\x1b[K',
 'remote: Counting objects:   7% (478/6819)\x1b[K',
 'remote: Counting objects:   8% (546/6819)\x1b[K',
 'remote: Counting objects:   9% (614/6819)\x1b[K',
 'remote: Counting objects:  10% (682/6819)\x1b[K',
 'remote: Counting objects:  11% (751/6819)\x1b[K',
 'remote: Counting objects:  12% (819/6819)\x1b[K',
 'remote: Counting objects:  13% (887/6819)\x1b[K',
 'remote: Counting objects:  14% (955/6819)\x1b[K',
 'remote: Counting objects:  15% (1023/6819)\x1b[K',
 'remote: Counting objects:  16% (1092/6819)\x1b[K',
 'remote: Counting objects:  17% 

In [4]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include 
I NVCCFLAGS: -std=c++11 -O3 -use_fast_math --forward-

In [5]:
from huggingface_hub import snapshot_download

In [11]:
model_name = "Qwen/Qwen1.5-1.8B"

In [7]:
methods=["q4_k_m"]

In [8]:
base_model= './original_model/'
quantized_path='./quantized_model/'

In [12]:
snapshot_download(repo_id=model_name,local_dir=base_model,local_dir_use_symlinks=False)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.28k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/content/original_model'

In [13]:
original_model= quantized_path+"/FB16.gguf"

In [14]:
!mkdir ./quantized_model/

In [19]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile  ./quantized_model/FP16.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
gguf: Adding 151387 merge(s).
gguf: Setting special token type eos to 151643
gguf: Setting special token type pad to 151643
gguf: Setting special token type bos to 151643
gguf: Setting chat_template to {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}
Exporting model to 'quantized_model/FP16.gguf'
gguf: loading model part 'model.safetensors'
output.weight, n_dims = 2, torch.bfloat16 --> float16
token_embd.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_gate.weight, n_dims = 2, t

## Type 0 Quant:

w=d*q

##Type 1 Quant:

w=d*q+m

In [17]:
import os

In [20]:
for m in methods:
  qtype=f"{quantized_path}/{m.upper()}.gguf"
  os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [24]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0  --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 2322 (fa974646)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709449002
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:        

In [32]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
from huggingface_hub import HfApi,HfFolder, create_repo, upload_file

In [33]:
model_path="./quantized_model/Q4_K_M.gguf"
repo_name ="qwen1.5-1.8B-GGUF"
repo_url= create_repo(repo_name, private=False)

In [34]:
api= HfApi()

In [37]:
api.upload_file(
  path_or_fileobj=model_path,
  path_in_repo= "Q4_K_M.gguf",
  repo_id= "kesavan1994/qwen1.5-1.8B-GGUF1" ,
  repo_type="model"
)

Q4_K_M.gguf:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kesavan1994/qwen1.5-1.8B-GGUF1/commit/0442e3a9ad96bf3375c8276e57388279339fc10c', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='0442e3a9ad96bf3375c8276e57388279339fc10c', pr_url=None, pr_revision=None, pr_num=None)

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [66]:
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/.git/


In [68]:
!git add "/content/drive/MyDrive/Colab Notebooks/Quantizing LLM using GGUF.ipynb"

In [69]:
!git commit -m "quantization using GGUF"

[master 743a510] quantization using GGUF
 1 file changed, 1 insertion(+), 1 deletion(-)


In [70]:
!git config --global user.email "keshavan1994@gmail.com"
!git config --global user.name "kesavan1994"

In [ ]:
!git status

In [71]:
!git remote add origin https://github.com/kesavan1994/quantization-using-GGUF.git

error: remote origin already exists.


In [72]:
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address
